In [53]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)



Toggle code

# Plot regional maps with mean current ellipses on top


In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable


import iris
import iris.plot as iplt
import iris.quickplot as qplt

import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


import palettable
from palettable import colorbrewer
from palettable.cmocean.sequential import Ice_12 as ocean_blue
from palettable.cmocean.sequential import Gray_12 as ocean_gray

plt.style.use('dark_background')
%matplotlib inline

#plt.rcParams.update({'font.size': 14})

## 1. Load data

### Load location of EEL stations

In [55]:
pathdir = '../data/raw/csv_ctdgrid'

file3 = pathdir+'/'+'EELCTDandLADCP_refpos.csv'
dfloc = pd.read_csv(file3,sep=',', index_col=None, 
                     header=0)
# Make sure the station name are sorted by their distance along the section
sdfloc = dfloc.sort_values('Refdist', ascending=True)
sdfloc['Refdist']=sdfloc['Refdist'].apply(int)
sdfloc

,Staname,Refdist,LonSta,LatSta,DepthSta
68,IB23S,0.000000,-20.215,63.317,120
67,IB22S,13.442044,-20.067,63.216,670
66,IB21S,25.375111,-19.916,63.133,1030
65,IB20S,55.616831,-19.551,62.917,1400
64,IB19S,84.025700,-19.668,62.667,1670
...,...,...,...,...,...
4,5G,1277.489387,-6.600,56.733,80
3,4G,1286.632474,-6.450,56.733,115
2,3G,1292.405649,-6.367,56.708,70
1,2G,1298.235343,-6.283,56.683,30


<br><br>
### Load LADCP Current data into a dataframe

In [265]:
file4 = pathdir+'/'+'EEL_LADCP_3Dfield.csv'
dflad = pd.read_csv(file4,sep=',', index_col=None, 
                     header=0)
dflad=dflad.round({'LADCP_U': 3, 'LADCP_V': 3})
dflad['Refdist']=dflad['Refdist'].apply(int)
dflad

,CruiseID,Year,Staname,Refdist,Depth,LADCP_U,LADCP_V
0,d22396,1996,13G,1191,25,-0.227,0.019
1,d22396,1996,13G,1191,35,-0.230,0.044
2,d22396,1996,13G,1191,45,-0.236,0.049
3,d22396,1996,13G,1191,55,-0.251,0.052
4,d22396,1996,13G,1191,65,-0.283,0.066
...,...,...,...,...,...,...,...
76029,dy078,2017,IB23S,0,65,0.026,0.078
76030,dy078,2017,IB23S,0,75,0.045,0.146
76031,dy078,2017,IB23S,0,85,0.036,0.166
76032,dy078,2017,IB23S,0,95,0.026,0.180


In [297]:
# Calculate depth average current for each year at each station using a pivot table
df_U = dflad.pivot_table(values='LADCP_U', index="Year", columns="Staname").round(3)
df_V = dflad.pivot_table(values='LADCP_V', index="Year", columns="Staname").round(3)
df_V.head()

Staname,10G,13G,14G,15G,8G,9G,A,B,C,D,...,L,M,N,O,P,Q,Q1,R,S,T
Year,,,,,,,,,,,,,,,,,,,,,
1996,NaN,0.055,0.08,0.116,NaN,NaN,-0.030,0.001,0.086,0.017,...,-0.105,-0.059,0.145,-0.013,0.083,0.222,0.168,0.118,0.120,0.118
1997,NaN,NaN,NaN,NaN,NaN,NaN,0.024,NaN,NaN,0.009,...,NaN,-0.053,-0.023,0.021,-0.001,0.106,0.047,0.199,NaN,-0.269
1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.187,...,NaN,-0.032,-0.044,-0.058,0.038,0.303,0.250,-0.131,NaN,NaN
1999,NaN,NaN,NaN,NaN,NaN,NaN,-0.035,-0.074,0.191,0.094,...,0.053,-0.187,-0.039,0.027,0.117,0.169,0.155,NaN,NaN,NaN
2000,-0.116,NaN,NaN,-0.056,NaN,-0.101,NaN,NaN,NaN,NaN,...,-0.016,0.003,-0.120,0.066,0.141,0.096,0.230,-0.126,-0.141,-0.004


In [192]:
def sortEELPV(dftosort, refdf=sdfloc):
    """ 
    Function to sort a dataframe where the rows correspond to Extend Ellet Line station name.
    The rows are sorted according to another dataframe containing the reference distance along 
    the EEL section associated to every station. Additional metadata corresponding to the reference station (lat, lon, depth)
    are also merged into the new dataframe.

    """
    dfnew=pd.merge(refdf,
                   dftosort,
                   how='left',
                   on='Staname').round(3)
    dfnew = dfnew.set_index('Staname',drop=True) 
    return dfnew


# Sort the dataframe row according to location of the station along the section.
# We use the reference dataframe sdfloc and merge additional metadata from sdfloc (lat, lon) into the new dataframe
df_U = sortEELPV(df_U.T, refdf=sdfloc)
df_V = sortEELPV(df_V.T, refdf=sdfloc)

# importing dataframe in xarray with Staname as index doesnt seem to work. Try here with Refdist
df_V = df_V.reset_index().set_index('Refdist')
df_U = df_U.reset_index().set_index('Refdist')

In [193]:
df_U

,Staname,LonSta,LatSta,DepthSta,1996,1997,1998,1999,2000,2005,2006,2009,2010,2011,2013,2014,2015,2016,2017
Refdist,,,,,,,,,,,,,,,,,,,
0,IB23S,-20.215,63.317,120,NaN,NaN,NaN,NaN,NaN,-0.113,0.076,NaN,0.106,-0.147,NaN,0.135,0.060,0.063,0.005
13,IB22S,-20.067,63.216,670,NaN,-0.001,NaN,-0.056,NaN,-0.064,-0.077,NaN,-0.106,-0.072,0.048,-0.057,-0.119,-0.069,-0.004
25,IB21S,-19.916,63.133,1030,NaN,NaN,NaN,NaN,NaN,-0.055,0.056,NaN,0.051,0.060,0.059,0.033,-0.135,-0.027,-0.046
55,IB20S,-19.551,62.917,1400,NaN,NaN,NaN,NaN,NaN,-0.003,0.091,NaN,-0.075,-0.136,-0.070,-0.078,0.003,-0.013,-0.127
84,IB19S,-19.668,62.667,1670,NaN,NaN,NaN,NaN,NaN,-0.179,-0.090,NaN,-0.092,-0.026,-0.126,-0.100,0.008,0.055,-0.106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1277,5G,-6.600,56.733,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1286,4G,-6.450,56.733,115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1292,3G,-6.367,56.708,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [194]:
df_V = df_V.drop(columns=['Staname', 'LonSta', 'LatSta', 'DepthSta'])
df_Vs=df_V.stack().rename_axis(['Refdist', 'Year']).rename('V')

df_U = df_U.drop(columns=['Staname', 'LonSta', 'LatSta', 'DepthSta'])
df_Us=df_U.stack().rename_axis(['Refdist', 'Year']).rename('U')


In [196]:
df_Us=pd.DataFrame(df_Us)

In [197]:
df=pd.concat([df_Us,df_Vs],axis=1)
df

U      V
Refdist Year              
0       2005 -0.113  0.182
        2006  0.076  0.059
        2010  0.106  0.011
        2011 -0.147 -0.024
        2014  0.135 -0.048
...             ...    ...
1232    2017  0.110  0.136
1242    2005  0.007  0.102
        2015 -0.141 -0.057
        2016 -0.214 -0.114
        2017  0.155  0.137

[621 rows x 2 columns]

\
#### Finally I just realised there are a much more quicker way using *groupby*.....

In [291]:
dfgb = dflad.groupby(by=['Refdist','Year']).mean().drop(columns='Depth').round(3).rename(columns={"LADCP_U": "U", "LADCP_V": "V"})

<br><br>
### Convert dataframe into xarray

In [292]:
da = dfgb.to_xarray()

In [293]:
da

<xarray.Dataset>
Dimensions:  (Refdist: 60, Year: 15)
Coordinates:
  * Refdist  (Refdist) int64 0 13 25 55 84 122 ... 1170 1180 1191 1222 1232 1242
  * Year     (Year) int64 1996 1997 1998 1999 2000 ... 2013 2014 2015 2016 2017
Data variables:
    U        (Refdist, Year) float64 nan nan nan nan ... nan -0.141 -0.214 0.155
    V        (Refdist, Year) float64 nan nan nan nan ... nan -0.057 -0.114 0.137

\
#### Quick test that the xarray is containing the same data than the dataframe
I had some problems earlier, when was of my dataframe index were strings (Station name) instead of numeric values... 

I changed the code now. I am dropping the Staname column and keep the refdist column for the xarray conversion. It seems to be working fine now...

In [294]:
ii = 40
da.U[ii].values

array([ 0.04 , -0.017, -0.119, -0.004,  0.077,  0.016,  0.193, -0.006,
       -0.019,    nan,  0.006, -0.041,  0.069, -0.029,  0.081])

In [295]:
da.U.Year.values

array([1996, 1997, 1998, 1999, 2000, 2005, 2006, 2009, 2010, 2011, 2013,
       2014, 2015, 2016, 2017])

In [296]:
df.loc[da.U.Refdist.values[ii]]['U']

Year
1996    0.040
1997   -0.017
1998   -0.119
1999   -0.004
2000    0.077
2005    0.016
2006    0.193
2009   -0.006
2010   -0.019
2013    0.006
2014   -0.041
2015    0.069
2016   -0.029
2017    0.081
Name: U, dtype: float64

#### TODO: plot distribution of depth average velocity at a specific station and compute the associated ellipse
Have a look to:
1. J. Lilly notes on variance ellipse http://jmlilly.net/course/v3/VarianceEllipsesDemo.html
2. Confidence ellipse function https://matplotlib.org/3.2.1/gallery/statistics/confidence_ellipse.html#sphx-glr-gallery-statistics-confidence-ellipse-py
3. How to draw patch using cartopy https://scitools.org.uk/cartopy/docs/v0.5/matplotlib/introductory_examples/02.polygon.html
